1. Import Necessary Libraries


In [32]:
# Import the conversational AI model and tokenizer from Hugging Face's Transformers library
from transformers import AutoModelForCausalLM,AutoTokenizer

# Import PyTorch for model usage
import torch

# Import tkinter library to create the GUI for the chatbot
import tkinter as tk

# Import scrolledtext for scrollable chat window and font for styling
from tkinter import scrolledtext,font

# Import time library for response delay
import time

# Import langdetect for detecting the user's language in messages
import langdetect

2. Load the Model and Tokenizer


In [33]:
# Define the pre-trained model name from Hugging Face's library (DialoGPT medium model)
model_name="microsoft/DialoGPT-medium"

# Load the tokenizer for the model
tokenizer=AutoTokenizer.from_pretrained(model_name)

# Load the conversational AI model
model=AutoModelForCausalLM.from_pretrained(model_name)

# Initialize a variable to store chat history
chat_history_ids=None

3. Language Detection Function


In [34]:
# Function to detect the language of the user's message
def detect_language(text):
    """Detects the language of the user's message."""
    try:
        # Use langdetect to determine the language code of the text
        lang=langdetect.detect(text)
        # Return English or Turkish based on the detected language
        if lang=="en":
            return "English"
        elif lang=="tr":
            return "Turkish"
        else:
            return "Unknown"
    except:
        # Return 'Unknown' if detection fails
        return "Unknown"

4. Generate Chatbot Response Function


In [35]:
# Function to generate a response from the chatbot based on the user's input
def get_chatbot_response(user_input):
    """Generates a response from the model based on user input."""
    global chat_history_ids
    # Encode the user's input and add an end-of-sequence token
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    # If chat history exists, concatenate with new input; otherwise, start a new chat
    bot_input_ids = new_user_input_ids if chat_history_ids is None else torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
    # Generate the bot's response based on the conversation context
    response = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    # Decode the response to readable text
    bot_response = tokenizer.decode(response[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    # Update chat history with the latest input and response
    chat_history_ids = bot_input_ids
    return bot_response

5. Send Message Function


In [36]:
# Function to display user's message and generate a bot response
def send_message(event=None):
    """Displays user's message and generates a bot response."""
    # Retrieve user's input from input field
    user_input = user_input_field.get()
    user_input_field.delete(0, tk.END)  # Clear the input field after getting the message

    # If user input is empty, ignore it
    if user_input.strip() == "":
        return

    # Limit message length to prevent long inputs
    if len(user_input) > 100:
        chat_window.configure(state='normal')
        chat_window.insert(tk.END, "Chatbot: Please keep your message under 100 characters.\n", 'bot')
        chat_window.configure(state='disabled')
        return

    # Detect the user's language
    user_language = detect_language(user_input)

    # Display the user's message in the chat window
    chat_window.configure(state='normal')
    chat_window.insert(tk.END, "User: " + user_input + '\n', 'user')
    chat_window.configure(state='disabled')

    # Display a "thinking" message from the bot while generating a response
    chat_window.configure(state='normal')
    chat_window.insert(tk.END, "Chatbot: ... (thinking)\n", 'bot')
    chat_window.configure(state='disabled')

    # Scroll to the bottom of the chat window
    chat_window.yview(tk.END)
    root.update_idletasks()  # Update the GUI for immediate feedback
    time.sleep(1)  # Simulate delay in response

    # Generate a bot response to the user's input
    bot_response = get_chatbot_response(user_input)

    # Display the bot's response in the chat window
    chat_window.configure(state='normal')
    chat_window.delete("end-1c")  # Remove "thinking" message
    chat_window.insert(tk.END, "Chatbot: " + bot_response + '\n', 'bot')
    chat_window.configure(state='disabled')
    chat_window.yview(tk.END)  # Scroll to bottom


6. Clear Chat Function


In [37]:
# Function to clear chat and reset the chat window with a welcome message
def clear_chat():
    """Clears the chat window and displays a welcome message."""
    chat_window.configure(state='normal')
    chat_window.delete(1.0, tk.END)  # Clear all text in chat window
    chat_window.insert(tk.END, "Chatbot: Hello! I am here to help. How can I assist you?\n", 'bot')
    chat_window.configure(state='disabled')


7. Initialize the Tkinter Window


In [38]:
# Initialize the main window for the application
root = tk.Tk()
root.title("Chatbot Application")  # Set the window title
root.geometry("500x600")  # Set the window size
root.resizable(False, False)  # Disable resizing


''

8. Configure Fonts


In [39]:
# Configure font settings for the chat window and input fields
style = font.Font(family="Helvetica", size=10)


9. Create the Chat Window


In [40]:
# Create a scrollable chat window where conversation history is displayed
chat_window = scrolledtext.ScrolledText(root, state='disabled', wrap=tk.WORD, font=style)
chat_window.grid(row=0, column=0, columnspan=2, padx=10, pady=10, sticky='nsew')


10. User Input Field


In [41]:
# Input field for the user to type messages
user_input_field = tk.Entry(root, width=40, font=style)
user_input_field.grid(row=1, column=0, padx=(10, 5), pady=10, sticky='ew')


11. Send and Clear Buttons


In [42]:
# Button to send messages
send_button = tk.Button(root, text="➤", command=send_message, font=style, bg="lightblue", width=3)
send_button.grid(row=1, column=1, padx=(5, 10), pady=10, sticky='w')

# Centered button to clear the chat history
clear_button = tk.Button(root, text="Clear", command=clear_chat, font=style, bg="lightcoral")
clear_button.grid(row=2, column=0, columnspan=2, pady=5)


12. Configure Tags for Text Colors


In [43]:
# Configure color for user and bot messages
chat_window.tag_config('user', foreground='blue')  # User messages in blue
chat_window.tag_config('bot', foreground='green')  # Bot messages in green


13. Display Initial Welcome Message


In [44]:
# Display an initial welcome message from the bot when the application starts
chat_window.configure(state='normal')
chat_window.insert(tk.END, "Chatbot: Hello! I am here to help. How can I assist you?\n", 'bot')
chat_window.configure(state='disabled')


14. Key Bindings and Start Application


In [45]:
# Bind the Enter key to the send message function
user_input_field.bind('<Return>', send_message)

# Start the main application loop
root.mainloop()
